In [2]:
import torch
import torchmetrics
torch.manual_seed(0)
torch.backends.cudnn.deterministic = False
torch.backends.cudnn.benchmark = True
import numpy as np
import torch.nn as nn
from torch.utils.data.dataset import Dataset

import pandas as pd

import tqdm, sys, os, gc, argparse, warnings

warnings.filterwarnings("ignore")

import gc
import time

if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [3]:
# path = "../data/"

# train_files = os.listdir(path + "train")
# train_df = pd.DataFrame()
# for filename in tqdm.tqdm(train_files):
#     tmp = pd.read_csv(path + "train/" + filename)
#     tmp["file"] = filename
#     train_df = pd.concat([train_df, tmp], axis=0, ignore_index=True)

# test_files = os.listdir(path + "test")
# test_df = pd.DataFrame()
# for filename in tqdm.tqdm(test_files):
#     tmp = pd.read_csv(path + "test/" + filename)
#     tmp["file"] = filename
#     test_df = pd.concat([test_df, tmp], axis=0, ignore_index=True)

In [4]:
# train_df["tick"] = (
#     (train_df["date"] * 24 + train_df["time"].apply(lambda x: int(x.split(":")[0])))
#     * 60
#     + train_df["time"].apply(lambda x: int(x.split(":")[1]))
# ) * 20 + train_df["time"].apply(lambda x: int(x.split(":")[2])) // 3
# test_df["tick"] = (
#     (test_df["date"] * 24 + test_df["time"].apply(lambda x: int(x.split(":")[0]))) * 60
#     + test_df["time"].apply(lambda x: int(x.split(":")[1]))
# ) * 20 + test_df["time"].apply(lambda x: int(x.split(":")[2])) // 3

# train_df = train_df.sort_values(["sym", "tick"])
# test_df = test_df.sort_values(["sym", "tick"])

# cols=['file' ,'uuid','n_close' ,'amount_delta', 'n_midprice', 'n_bid1', 'n_bsize1', 'n_bid2', 'n_bsize2', 'n_ask1', 'n_asize1', 'n_ask2', 'n_asize2']
# target_df=train_df[['label_5','label_10','label_20','label_40','label_60']]
# train_df=train_df[cols]
# test_df=test_df[cols]

In [5]:
# target_df.to_csv('.\data\\target.csv',index=False)
# train_df.to_csv('.\data\\train.csv',index=False)
# test_df.to_csv('.\data\\test.csv',index=False)

In [6]:
# train_df=pd.read_csv('.\data\\train.csv')
# target_df=pd.read_csv('.\data\\target.csv')
# test_df=pd.read_csv('./data//test.csv')

In [7]:
path = "../data/"
train_df=pd.read_csv(path+'train/'+'snapshot_sym0_date0_am.csv')
test_df=pd.read_csv(path+'test/'+'snapshot_sym0_date64_am.csv')

cols=['n_close' ,'amount_delta', 'n_midprice', 'n_bid1', 'n_bsize1', 'n_bid2', 'n_bsize2', 'n_ask1', 'n_asize1', 'n_ask2', 'n_asize2']
target_df=train_df[['label_5','label_10','label_20','label_40','label_60']]
train_df=train_df[cols]
test_df=test_df[cols]
train_df=train_df.astype('float32')
test_df=test_df.astype('float32')

In [8]:
seq_length=50
tmp=train_df.iloc[[0]]
for i in range(seq_length-2):
    tmp=tmp.append(train_df.iloc[[0]])
train_df=tmp.append(train_df)
tmp=test_df.iloc[[0]]
for i in range(seq_length-1):
    tmp=tmp.append(test_df.iloc[[0]])
test_df=tmp.append(test_df)

In [9]:
features=torch.from_numpy(np.array(train_df))
# features=features.permute(1,0)
# features.size()
features = features.unfold(0,50,1)
# features.size()
features=features.permute(0,2,1)
features.size()

torch.Size([1999, 50, 11])

In [39]:
targets=torch.from_numpy(np.array(target_df))
targets[1]

tensor([0, 0, 0, 0, 0])

In [38]:
seq_length=50
features=torch.tensor([])
targets=torch.tensor([])
tfeatures=torch.tensor([])
for i in range(1999):
    f=torch.from_numpy(np.array(train_df.iloc[i:i+seq_length]))
    tf=torch.from_numpy(np.array(test_df.iloc[i:i+seq_length]))
    t=torch.from_numpy(np.array(target_df.iloc[[i]]))
    features=torch.cat((features,torch.unsqueeze(f,0)),0)
    tfeatures=torch.cat((tfeatures,torch.unsqueeze(tf,0)),0)
    targets=torch.cat((targets,t),0)
# features[0]
targets[1]

tensor([0., 0., 0., 0., 0.])

In [9]:
target_index=0

In [10]:
class SeqDataset(Dataset):
    def __init__(self, data_features, data_targets,target_index):
        self.len = data_features.size()[0]
        self.features=data_features
        self.targets=data_targets[:,target_index]
    def __getitem__(self, index):
        return self.features[index], self.targets[index]
    def __len__(self):
        return self.len

Data_set=SeqDataset(data_features=features,data_targets=targets,target_index=target_index)
test_set=SeqDataset(data_features=tfeatures,data_targets=targets,target_index=target_index)
train_dataset, val_dataset = torch.utils.data.random_split(
    dataset=Data_set, lengths=[0.9, 0.1], generator=torch.Generator().manual_seed(42)
)

In [11]:
trainLoader=torch.utils.data.DataLoader(train_dataset,batch_size=16,shuffle=True,num_workers=0, pin_memory=False)
valLoader=torch.utils.data.DataLoader(val_dataset,batch_size=16,shuffle=True,num_workers=0, pin_memory=False)
testLoader=torch.utils.data.DataLoader(test_set,batch_size=32,shuffle=True,num_workers=0, pin_memory=False)

In [12]:
# GRU
class GRU(nn.Module):
    def __init__(self):
        super(GRU, self).__init__()
        self.gru = nn.GRU(
            input_size=len(cols),
            hidden_size=64,
            batch_first=True,
        )
        self.mlp = nn.Sequential(
            nn.Linear(64, 32),
            nn.LeakyReLU(),
            nn.Linear(32, 16),
            nn.LeakyReLU(),
            nn.Linear(16, 3),
        )
    
    def forward(self, input):
        output, _ = self.gru(input, None)
        output = output[:, -1, :]
        output = self.mlp(output)
        return output

model=GRU().to(device)
optimizer = torch.optim.AdamW(model.parameters(), 0.0001)
criterion = nn.CrossEntropyLoss().cuda()

In [13]:
def train(train_loader, model, criterion, optimizer):
    start = time.time()
    start_batch = [start, 0]
    model.train()
    train_loss = 0.0
    preds = torch.tensor([])
    target_all = torch.tensor([])
    for i, (input, target) in enumerate(train_loader):
        input = input.to(device)
        target = target.to(torch.int64).to(device)
        output = model(input)
        loss = criterion(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if (i + 1) % 100 == 0:
            start_batch[((i + 1) // 100) % 2] = time.time()
            print(
                "Train loss",
                loss.item(),
                "t={}s".format(
                    start_batch[((i + 1) // 100) % 2]
                    - start_batch[(1 + (i + 1) // 100) % 2]
                ),
            )

        preds = torch.cat((preds, output.cpu().argmax(1)))
        target_all = torch.cat((target_all, target.cpu()))

        train_loss += loss.item()

    val_acc = torchmetrics.functional.classification.multiclass_f1_score(
        preds, target_all, num_classes=25, average="macro"
    )
    print(i)
    print("t={}s".format(time.time() - start))
    print("F1 score", val_acc)
    return train_loss / len(train_loader)

def validate(val_loader, model, criterion):
    model.eval()
    val_acc = 0.0
    preds = torch.tensor([])
    target_all = torch.tensor([])
    val_loss=0.0
    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):
            input = input.to(device)
            target = target.to(torch.int64).to(device)
            output = model(input)
            loss = criterion(output, target)
            # val_acc += (output.argmax(1) == target).sum().item()
            preds = torch.cat((preds, output.cpu().argmax(1)))
            target_all = torch.cat((target_all, target.cpu()))

        val_acc = torchmetrics.functional.classification.multiclass_f1_score(
            preds, target_all, num_classes=25, average="macro"
        )
        val_loss += loss.item()
    # return val_acc / len(val_loader.dataset)
    return val_acc,val_loss/len(val_loader)

def predict(test_loader, model):
    model.eval()

    test_pred = []
    with torch.no_grad():
        for i, (input, _) in enumerate(test_loader):
            input = input.to(device)
            output = model(input)
            test_pred.append(output.data.cpu().numpy())

    return np.vstack(test_pred)

In [28]:
epochs=50
for epoch in range(epochs):
    train_loss = train(trainLoader, model, criterion, optimizer)
    val_acc,val_loss = validate(valLoader, model, criterion)
    print(
        "epoch {} :".format(epoch + 1), "train_loss:", train_loss, 'val_loss:', val_loss,"val_f1:", val_acc
    )
    # epochs += 1
# torch.save(model.state_dict(),"./model/baseline_test.pth")

Train loss 0.4153522849082947 t=0.4659888744354248s
112
t=0.4960815906524658s
F1 score tensor(0.0339)
epoch 1 : train_loss: 0.7594163502212119 val_loss: 0.08051744791177604 val_f1: tensor(0.0349)
Train loss 0.842707633972168 t=0.23580670356750488s
112
t=0.2806997299194336s
F1 score tensor(0.0339)
epoch 2 : train_loss: 0.7602404590729064 val_loss: 0.06137483853560228 val_f1: tensor(0.0349)
Train loss 0.531100869178772 t=0.22405719757080078s
112
t=0.25747036933898926s
F1 score tensor(0.0339)
epoch 3 : train_loss: 0.7611211406446137 val_loss: 0.06077424837992741 val_f1: tensor(0.0349)
Train loss 0.30872175097465515 t=0.22574877738952637s
112
t=0.2560088634490967s
F1 score tensor(0.0339)
epoch 4 : train_loss: 0.7597316669679322 val_loss: 0.08009287027212289 val_f1: tensor(0.0349)
Train loss 0.8356626629829407 t=0.21640753746032715s
112
t=0.2416379451751709s
F1 score tensor(0.0339)
epoch 5 : train_loss: 0.759563180461394 val_loss: 0.06206699059559749 val_f1: tensor(0.0349)
Train loss 1.0694

In [29]:
pred = predict(testLoader, model)
pred


array([[-0.74168557,  0.99619937, -0.84154886],
       [-0.74168557,  0.99619937, -0.84154886],
       [-0.74168557,  0.99619937, -0.84154886],
       ...,
       [-0.74168557,  0.99619925, -0.84154874],
       [-0.74168557,  0.99619925, -0.84154874],
       [-0.74168557,  0.99619925, -0.84154874]], dtype=float32)

In [26]:
a=torch.tensor([])
b=torch.from_numpy(np.array(train_df.iloc[[0]]))
torch.cat((a,b),0)

tensor([[1.5691e-02, 2.5949e+06, 1.5270e-02, 1.5130e-02, 1.9615e-06, 1.4850e-02,
         9.9739e-07, 1.5410e-02, 2.6597e-07, 1.5691e-02, 3.4576e-06]],
       dtype=torch.float64)

In [ ]:
a=torch.tensor([])
b=torch.tensor([1])
torch.cat((a,b),0).size()

In [ ]:
i=0
seq_length=50
f=torch.from_numpy(np.array(train_df.iloc[i+seq_length:i+2*seq_length]))
f=torch.unsqueeze(f,0)
f.size()